# A Tour of `ganda`

This user guide is built in an interactive `bash` Jupyter Notebook.  If you've got `ganda` [installed](../README.md#installation) and in your `PATH` you can run the same commands.

In [1]:
# ensure that the ganda executable is in your PATH
which ganda >/dev/null && echo "ganda found in PATH" || echo "ganda not found in PATH"

ganda found in PATH


# `ganda` Usage

In [2]:
ganda help

NAME:
   ganda - make http requests in parallel

USAGE:
   <urls/requests on stdout> | ganda [options]

VERSION:
   1.0.0 80d3f4e 2024-06-28

DESCRIPTION:
   Pipe urls to ganda over stdout for it to make http requests to each url in parallel.

AUTHOR:
   Ted Naleid <contact@naleid.com>

COMMANDS:
   echoserver  Starts an echo server, --port <port> to override the default port of 8080
   help, h     Shows a list of commands or help for one command

GLOBAL OPTIONS:
   --base-retry-millis value                              the base number of milliseconds to wait before retrying a request, exponential backoff is used for retries (default: 1000)
   --response-body value, -B value                        transforms the body of the response. Values: 'raw' (unchanged), 'base64', 'discard' (don't emit body), 'escaped' (JSON escaped string), 'sha256' (default: raw)
   --connect-timeout-millis value                         number of milliseconds to wait for a connection to be established before ti

# `ganda` Basics


`ganda` makes HTTP requests, similar to `curl`, just pipe it an URL on stdin and it will make a `GET` request and echo the body of the response on stdout.  The status code of the URL will be sent to stderr.

We'll use [httpbin.org](http://httpbin.org) for the first few requests.  It returns a JSON representation of the request in the body of the response.

In [3]:
echo "http://httpbin.org/anything/1" | ganda 

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept-Encoding": "gzip", 
    "Host": "httpbin.org", 
    "User-Agent": "Go-http-client/1.1", 
    "X-Amzn-Trace-Id": "Root=1-66a7ddb6-11dee20d771e068732f77d58"
  }, 
  "json": null, 
  "method": "GET", 
  "origin": "173.16.32.166", 
  "url": "http://httpbin.org/anything/1"
}
Response: 200 http://httpbin.org/anything/1



You can pipe multiple URLs to `ganda`.  It happily lives in the middle of shell pipes for making requests.

Here we make 3 requests to `/anything/1`, `/anything/2`, and `/anything/3` and pipe them to `jq` where we grab just the `method` and `url` properties from the response.

We've also added the `-s` (silent) flag to `ganda` to suppress the stderr output that shows the url and response codes.

In [4]:
seq 3 |\
  awk '{printf "http://httpbin.org/anything/%s\n", $1}' |\
  ganda -s |\
  jq -c '{method, url}'

{"method":"GET","url":"http://httpbin.org/anything/1"}
{"method":"GET","url":"http://httpbin.org/anything/2"}
{"method":"GET","url":"http://httpbin.org/anything/3"}


## JSON Output

`ganda` uses the `-J` flag for JSON output.  This emits JSON with the response as the `"body"` field and includes other details about the request:

In [5]:
echo "http://httpbin.org/anything/1" |\
  ganda -s -J |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": {
    "args": {},
    "data": "",
    "files": {},
    "form": {},
    "headers": {
      "Accept-Encoding": "gzip",
      "Host": "httpbin.org",
      "User-Agent": "Go-http-client/1.1",
      "X-Amzn-Trace-Id": "Root=1-66a7ddb7-6652a41f4f1185c4563dc2eb"
    },
    "json": null,
    "method": "GET",
    "origin": "173.16.32.166",
    "url": "http://httpbin.org/anything/1"
  }
}


The body of the response is assumed to be JSON as a default.  This emits the `raw` response bytes after the `"body"` property.  If the response isn't JSON, you've got a few options for escaping/encoding the response using the `-B/--response-body <value>` flag:

1. `raw` - the default, shown above
2. `base64` - encode the bytes as a `base64` string, useful for binary content.
3. `discard` - drop the bytes and set the body to `null` 
4. `escaped` - escape the JSON and emit the value as a String
5. `sha256` - calculate the sha256 value of the body, useful for checking if the response has changed

In [6]:
# base64 encode the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B base64 |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": "ewogICJhcmdzIjoge30sIAogICJkYXRhIjogIiIsIAogICJmaWxlcyI6IHt9LCAKICAiZm9ybSI6IHt9LCAKICAiaGVhZGVycyI6IHsKICAgICJBY2NlcHQtRW5jb2RpbmciOiAiZ3ppcCIsIAogICAgIkhvc3QiOiAiaHR0cGJpbi5vcmciLCAKICAgICJVc2VyLUFnZW50IjogIkdvLWh0dHAtY2xpZW50LzEuMSIsIAogICAgIlgtQW16bi1UcmFjZS1JZCI6ICJSb290PTEtNjZhN2RkYjgtMTcwOTlhZDE1YzM4ZGViMTdhMzIxNzM2IgogIH0sIAogICJqc29uIjogbnVsbCwgCiAgIm1ldGhvZCI6ICJHRVQiLCAKICAib3JpZ2luIjogIjE3My4xNi4zMi4xNjYiLCAKICAidXJsIjogImh0dHA6Ly9odHRwYmluLm9yZy9hbnl0aGluZy8xIgp9Cg=="
}


In [7]:
# discard the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B discard |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": null
}


In [8]:
# JSON escape the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B escaped |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": "{\n  \"args\": {}, \n  \"data\": \"\", \n  \"files\": {}, \n  \"form\": {}, \n  \"headers\": {\n    \"Accept-Encoding\": \"gzip\", \n    \"Host\": \"httpbin.org\", \n    \"User-Agent\": \"Go-http-client/1.1\", \n    \"X-Amzn-Trace-Id\": \"Root=1-66a7ddb9-4f108a2f1d7000086a56f91d\"\n  }, \n  \"json\": null, \n  \"method\": \"GET\", \n  \"origin\": \"173.16.32.166\", \n  \"url\": \"http://httpbin.org/anything/1\"\n}\n"
}


In [9]:
# calculate the sha256 hash of the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B sha256 |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": "2256b295f3e4000cdd830da58d4aea30e30c4cc9cbd6c795512863bb5856f031"
}


## Customizing Requests with JSON Request Syntax

`ganda` supports an alternate JSON-lines syntax for requests.  The [JSON schema](../request.schema.json) is available, but the summary of fields it allows is:
- `"url"` - required string - is the only required field - the request URL
- `"method"` - optional string - a valid HTTP method (`GET|PUT|POST|DELETE|...`) - defaults to `GET`
- `"headers"` - optional JSON object - string key/value pairs
- `"context"` - optional JSON value - carried forward into the JSON output of the response, used to correlate requests and responses, can be a string, array, or object
- `"body"` - optional JSON value - a string or valid JSON object, the body of the request
- `"bodyType"` - optional enum - one of: `json` (default), `escaped`, or `base64`

### Adding a Request Body

What if you want to `POST` instead of `GET`?  `ganda` supports the same `-X <http method>` syntax that `curl` uses:

In [10]:
seq 3 |\
  awk '{printf "http://httpbin.org/anything/%s\n", $1}' |\
  ganda -s -X POST |\
  jq -c '{method, url}'

{"method":"POST","url":"http://httpbin.org/anything/1"}
{"method":"POST","url":"http://httpbin.org/anything/2"}
{"method":"POST","url":"http://httpbin.org/anything/3"}


But, along with most `POST` requests, you'll want to include a body.  `ganda` has an alternate JSON-lines syntax for requests that allows specifying the method and body:


In [11]:
echo '
{ 
  "method": "POST", 
  "url": "http://httpbin.org/anything/1", 
  "body": { "key1": "value1", "key2": "value2" } 
}' |\
  # ganda wants JSON-lines input, use jq -c to compact the JSON to a single line
  jq -c '.' |\
  ganda -s |\
  jq '.'

{
  "args": {},
  "data": "{\"key1\":\"value1\",\"key2\":\"value2\"}",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Content-Length": "33",
    "Host": "httpbin.org",
    "User-Agent": "Go-http-client/1.1",
    "X-Amzn-Trace-Id": "Root=1-66a7ddbd-542ec6147d8dde1778d30bb6"
  },
  "json": {
    "key1": "value1",
    "key2": "value2"
  },
  "method": "POST",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


By default, it assumes that the body in the request JSON is also a valid JSON object.  If you've got escaped JSON or base64-encoded binary data, you can use the optional `"bodyType"` JSON field to tell `ganda` to transform your input before sending it.

In [12]:
# "bodyType": "escaped" - ganda will unescape before making the request
echo '
{ 
  "url": "http://httpbin.org/anything/1", 
  "bodyType": "escaped", 
  "body": "{ \"key1\": \"value1\" }" 
}' |\
  jq -c '.' |\
  ganda -s -X POST |\
  jq '.'

{
  "args": {},
  "data": "{ \"key1\": \"value1\" }",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Content-Length": "20",
    "Host": "httpbin.org",
    "User-Agent": "Go-http-client/1.1",
    "X-Amzn-Trace-Id": "Root=1-66a7ddbe-1e2329056317fe8129273853"
  },
  "json": {
    "key1": "value1"
  },
  "method": "POST",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


In [13]:
# "bodyType": "base64" - ganda will decode before making the request
# generated with: echo -n '{ "value": "was base64 escaped" }' | base64
echo '
{ 
  "url": "http://httpbin.org/anything/1", 
  "bodyType": "base64", 
  "body": "eyAidmFsdWUiOiAid2FzIGJhc2U2NCBlc2NhcGVkIiB9" 
}' |\
  jq -c '.' |\
  ganda -s -X POST |\
  jq '.'

{
  "args": {},
  "data": "{ \"value\": \"was base64 escaped\" }",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "Transfer-Encoding": "chunked",
    "User-Agent": "Go-http-client/1.1",
    "X-Amzn-Trace-Id": "Root=1-66a7ddbe-589bcd0e5b7ce26e401337ef"
  },
  "json": {
    "value": "was base64 escaped"
  },
  "method": "POST",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


### Request Headers

`ganda` allows adding static headers to every request with the `-H key:value` syntax.  Multiple headers can be specified, and they'll override defaulted values:

In [14]:
echo '{ "url": "http://httpbin.org/anything/1" }' |\
  ganda -s -H "X-My-Header: 1234" -H "User-Agent: static-ganda" |\
  jq '.'

{
  "args": {},
  "data": "",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "User-Agent": "static-ganda",
    "X-Amzn-Trace-Id": "Root=1-66a7ddbf-540fe323135fefe5012a97cd",
    "X-My-Header": "1234"
  },
  "json": null,
  "method": "GET",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


The JSON-lines syntax can also specify per-request headers that will override static headers.

Here, the `User-Agent` in the JSON overrides the static header `User-Agent` from the `-H` flag:

In [15]:
echo '
{ 
  "url": "http://httpbin.org/anything/1", 
  "headers": {"X-Second-Header": "5678", "User-Agent": "per-request-ganda" } 
}' |\
  jq -c '.' |\
  ganda -s -H "X-My-Header: 1234" -H "User-Agent: static-ganda" |\
  jq '.'

{
  "args": {},
  "data": "",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "User-Agent": "per-request-ganda",
    "X-Amzn-Trace-Id": "Root=1-66a7ddbf-7bf3b5ae1e391b9110753ad7",
    "X-My-Header": "1234",
    "X-Second-Header": "5678"
  },
  "json": null,
  "method": "GET",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


## Request Context

Your requests are part of a pipeline, what if you want to carry context through your pipeline that isn't part of the HTTP request/response?

An example would be calling an HTTP endpoint to generate a new UUID, but the response does not include the ID that we want to associate with the UUID.

`ganda` allows you to specify values along with the URL that will still be present in the JSON envelope output.

This can be done with the simple request syntax by specifying tab-separated values after the URL:


In [16]:
# echo can emit tab separated values for correlating requests and responses
# here is what is being passed to ganda:
echo -e 'http://httpbin.org/uuid\t1\t"single\tvalue\twith\ttabs"'

http://httpbin.org/uuid	1	"single	value	with	tabs"


In [17]:
echo -e 'http://httpbin.org/uuid\t1\t"single\tvalue\twith\ttabs"' |\
  ganda -s -J |\
  jq '.'

{
  "url": "http://httpbin.org/uuid",
  "code": 200,
  "body": {
    "uuid": "9d5c32f1-2f53-4adc-baa5-27924f680b16"
  },
  "context": [
    "1",
    "single\tvalue\twith\ttabs"
  ]
}


Notice the `"context"` emitted at the bottom of the JSON.

The JSON-lines request format also allows context to be specified, and it can be any valid JSON object (string, array, or object):

In [18]:
echo '
{ 
  "url": "http://httpbin.org/uuid", 
  "context": { "id": 1, "value": "correlation value"} 
}' |\
  jq -c '.' |\
  ganda -s -J |\
  jq '.'

{
  "url": "http://httpbin.org/uuid",
  "code": 200,
  "body": {
    "uuid": "f62a0b7b-4136-4b62-ad6e-e90bd4e3898a"
  },
  "context": {
    "id": 1,
    "value": "correlation value"
  }
}


## `ganda echoserver` - a simple server that echoes requests 

`ganda` comes with a built-in echo server to make verifying requests easier. 

We don't want to hammer the public `httpbin.org` server, so let's fire up `ganda echoserver` as a background process and use that instead. 

In [19]:
ganda echoserver --help

NAME:
   ganda echoserver - Starts an echo server, --port <port> to override the default port of 8080

USAGE:
   ganda echoserver [command [command options]] 

OPTIONS:
   --port value          Port number to start the echo server on (default: 8080)
   --delay-millis value  Number of milliseconds to delay responding (default: 0)
   --help, -h            show help (default: false)


Normally, we'd run `ganda echoserver` in another terminal window with a command like:

```
ganda echoserver --port 9090
``` 
For this notebook, we'll run the echoserver in the background with a 1 second delay on every response.  We'll also suppress its logging output to stdout:

In [20]:
# run the echoserver in the background. give it a 1000ms/1s delay for responding to each request.  
# If you're running it in a separate terminal, you can omit the `>/dev/null &` part
ganda echoserver --port 9090 --delay-millis 1000 >/dev/null &

[1] 75816


Let's use `ganda` to make a single request to our echoserver so we can see its output.  It takes about a second because of the echoserver delay.

In [21]:
time echo "http://localhost:9090/anything/1" | ganda -s | jq '.'

{
  "time": "2024-07-29T13:21:55-05:00",
  "id": "",
  "remote_ip": "::1",
  "host": "localhost:9090",
  "method": "GET",
  "uri": "/anything/1",
  "user_agent": "Go-http-client/1.1",
  "status": 200,
  "headers": {
    "Accept-Encoding": "gzip",
    "Connection": "keep-alive",
    "User-Agent": "Go-http-client/1.1"
  },
  "request_body": ""
}

real	0m1.013s
user	0m0.021s
sys	0m0.006s


## Parallelizing Requests

By default, `ganda` using a single worker thread and a single connection to make requests.  This will guarantee that requests are made in the order they are received. 

If order doesn't matter, and you'd like to increase throughput, we can use the `-W <number of workers>` command.

In [22]:
# our echoserver is running with a 1000ms delay.  about 10 seconds to complete with the single default worker
seq 10 |\
  awk '{printf "http://localhost:9090/slow-api/%s\n", $1}' |\
  ganda |\
  # use pv - pipeviewer - to show the total number of requests, the total time taken, and the rate of requests per second
  pv -albert > /dev/null

Response: 200 http://localhost:9090/slow-api/1
Response: 200 http://localhost:9090/slow-api/2
Response: 200 http://localhost:9090/slow-api/3
Response: 200 http://localhost:9090/slow-api/4
Response: 200 http://localhost:9090/slow-api/5
Response: 200 http://localhost:9090/slow-api/6
Response: 200 http://localhost:9090/slow-api/7
Response: 200 http://localhost:9090/slow-api/8
Response: 200 http://localhost:9090/slow-api/9
Response: 200 http://localhost:9090/slow-api/10
10.0  0:00:10 [ 996m/s] [ 996m/s]


If we increase the number of workers to 10, we should finish in about a second

In [23]:
# our echoserver is running with a 1 second delay so the 10 requests 
# should be handled by 10 workers in about 1 second
seq 10 |\
  awk '{printf "http://localhost:9090/slow-api/%s\n", $1}' |\
  ganda -W 10 |\
  pv -albert > /dev/null

Response: 200 http://localhost:9090/slow-api/2
Response: 200 http://localhost:9090/slow-api/7
Response: 200 http://localhost:9090/slow-api/5
Response: 200 http://localhost:9090/slow-api/3
Response: 200 http://localhost:9090/slow-api/9
Response: 200 http://localhost:9090/slow-api/8
Response: 200 http://localhost:9090/slow-api/1
Response: 200 http://localhost:9090/slow-api/10
Response: 200 http://localhost:9090/slow-api/4
Response: 200 http://localhost:9090/slow-api/6
10.0  0:00:01 [9.78 /s] [9.78 /s]


In [24]:
# if we use 100 parallel workers and make 1k requests that 
# each take 1 second, it should take about 10 seconds
seq 1000 |\
  awk '{printf "http://localhost:9090/slow-api/%s\n", $1}' |\
  ganda -s -W 100 | 
  pv -albert > /dev/null

1.00k 0:00:10 [99.1 /s] [99.1 /s]


`ganda` also supports throttling the number of requests its workers will make using the `--throttle <requests per second>` flag.

If we use 100 parallel workers, but throttle them so that they can only make 5 requests per second, it should take about 20 seconds to complete 100 requests.

In [25]:
seq 100 |\
  awk '{printf "http://localhost:9090/slow-api/%s\n", $1}' |\
  ganda -s -W 100 --throttle 5 |\
  pv -albert > /dev/null

 100  0:00:21 [4.76 /s] [4.76 /s]


In [26]:
# clean up the delay echoserver that we'd previously run in the background
pkill ganda && echo "echoserver stopped" || echo "echoserver not stopped"

echoserver stopped


## Saving Individual Responses to Files

`ganda` also supports saving individual responses to files. This can be useful for debugging or for saving responses for later analysis. 

The `-o <directory_name>` flag is used to specify the directory where the responses should be saved.

In [27]:
# start up a echoserver that has no delay on port 9090 and put it in the background
ganda echoserver --port 9090 >/dev/null &

[1] 75847


Let's make 10 requests and save them as individual files in the `scratch/ten` directory

In [28]:
seq 10 |\
  awk '{printf "http://localhost:9090/fast-api/%s\n", $1}' |\
  ganda -W 1 -o scratch/ten |\
  pv -albert > /dev/null

Response: 200 http://localhost:9090/fast-api/1 -> scratch/ten/http-localhost-9090-fast-api-1
Response: 200 http://localhost:9090/fast-api/2 -> scratch/ten/http-localhost-9090-fast-api-2
Response: 200 http://localhost:9090/fast-api/3 -> scratch/ten/http-localhost-9090-fast-api-3
Response: 200 http://localhost:9090/fast-api/4 -> scratch/ten/http-localhost-9090-fast-api-4
Response: 200 http://localhost:9090/fast-api/5 -> scratch/ten/http-localhost-9090-fast-api-5
Response: 200 http://localhost:9090/fast-api/6 -> scratch/ten/http-localhost-9090-fast-api-6
Response: 200 http://localhost:9090/fast-api/7 -> scratch/ten/http-localhost-9090-fast-api-7
Response: 200 http://localhost:9090/fast-api/8 -> scratch/ten/http-localhost-9090-fast-api-8
Response: 200 http://localhost:9090/fast-api/9 -> scratch/ten/http-localhost-9090-fast-api-9
Response: 200 http://localhost:9090/fast-api/10 -> scratch/ten/http-localhost-9090-fast-api-10
0.00  0:00:00 [0.00 /s] [0.00 /s]


In [29]:
ls scratch/ten

http-localhost-9090-fast-api-1	http-localhost-9090-fast-api-5
http-localhost-9090-fast-api-10	http-localhost-9090-fast-api-6
http-localhost-9090-fast-api-2	http-localhost-9090-fast-api-7
http-localhost-9090-fast-api-3	http-localhost-9090-fast-api-8
http-localhost-9090-fast-api-4	http-localhost-9090-fast-api-9


In [30]:
# show the JSON envelope response for one of the requests
cat scratch/ten/http-localhost-9090-fast-api-1 | jq '.'

{
  "time": "2024-07-29T13:22:39-05:00",
  "id": "",
  "remote_ip": "::1",
  "host": "localhost:9090",
  "method": "GET",
  "uri": "/fast-api/1",
  "user_agent": "Go-http-client/1.1",
  "status": 200,
  "headers": {
    "Accept-Encoding": "gzip",
    "Connection": "keep-alive",
    "User-Agent": "Go-http-client/1.1"
  },
  "request_body": ""
}


That's great for a low number of files, but filesystems get cranky when you get more than low thousands of files in a single directory.

`ganda` supports a `--subdir-length <length>/-S <length>` flag that will hash the url and put the response in a subdirectory for that hash.

So with a `--subdir-length 2` the `http://localhost:9090/fast-api/1` response gets hashed to the `d8` subdirectory under our `scratch/ten-subdir-length-two` output directory.

In [31]:
seq 10 |\
  awk '{printf "http://localhost:9090/fast-api/%s\n", $1}' |\
  ganda -W 10 -o scratch/ten-subdir-length-two --subdir-length 2 |\
  pv -albert > /dev/null

Response: 200 http://localhost:9090/fast-api/6 -> scratch/ten-subdir-length-two/3f/http-localhost-9090-fast-api-6
Response: 200 http://localhost:9090/fast-api/9 -> scratch/ten-subdir-length-two/91/http-localhost-9090-fast-api-9
Response: 200 http://localhost:9090/fast-api/2 -> scratch/ten-subdir-length-two/1d/http-localhost-9090-fast-api-2
Response: 200 http://localhost:9090/fast-api/8 -> scratch/ten-subdir-length-two/d1/http-localhost-9090-fast-api-8
Response: 200 http://localhost:9090/fast-api/3 -> scratch/ten-subdir-length-two/55/http-localhost-9090-fast-api-3
Response: 200 http://localhost:9090/fast-api/5 -> scratch/ten-subdir-length-two/44/http-localhost-9090-fast-api-5
Response: 200 http://localhost:9090/fast-api/1 -> scratch/ten-subdir-length-two/d8/http-localhost-9090-fast-api-1
Response: 200 http://localhost:9090/fast-api/4 -> scratch/ten-subdir-length-two/47/http-localhost-9090-fast-api-4
Response: 200 http://localhost:9090/fast-api/7 -> scratch/ten-subdir-length-two/80/http-

Let's make 100k requests and see how many of them get hashed to the `d8` subdirectory that the `/fast-api/1` request hashed to above.

In [32]:
seq 100000 |\
  awk '{printf "http://localhost:9090/fast-api/%s\n", $1}' |\
  ganda -W 3 -o scratch/10k-subdir-length-two --subdir-length 2 2>&1 |\
  pv -albert > /dev/null

 100k 0:00:12 [7.73k/s] [7.73k/s]


So it took less than 15 seconds on my machine to make 100k requests to the echoserver and save each response to its own file.

In [33]:
# how many subdirectories were created
ls scratch/10k-subdir-length-two | wc -l        

     256


In [34]:
# show the first 5 subdirectories - expect 00, 01, 02, 03, 04
ls scratch/10k-subdir-length-two | head -n 5    

00
01
02
03
04


In [35]:
# how many files are in the d8 subdirectory
ls scratch/10k-subdir-length-two/d8 | wc -l

     406


In [36]:
# show the first 5 files in the d8 subdirectory
ls scratch/10k-subdir-length-two/d8 | head -n 5

http-localhost-9090-fast-api-1
http-localhost-9090-fast-api-1000
http-localhost-9090-fast-api-10057
http-localhost-9090-fast-api-10125
http-localhost-9090-fast-api-10135


There are 256 subdirectories, so every hash value was hit, and our sample subdirectory had 406 files in it.  A nice distribution.

In [37]:
# clean up the scratch directory
rm -rf scratch

# clean up the background echoserver
pkill ganda && echo "echoserver stopped" || echo "echoserver not stopped"

echoserver stopped


That's it!  A whirlwind tour of what `ganda` can do.